# Simple Kafka Consumer Test

This notebook connects to a Kafka broker and prints out every message it receives.

## Configuration
- **Bootstrap Servers**: Configure your Kafka broker address
- **Topic**: Specify the topic to consume from
- **Group ID**: Consumer group identifier

## Usage
1. Update the configuration variables below
2. Run all cells to start consuming messages
3. Messages will be printed in real-time
4. Use Ctrl+C in the cell to stop consumption

In [1]:
# Install required packages if not already installed
!pip install kafka-python


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [2]:
from kafka import KafkaConsumer
import json
from datetime import datetime
import time

# Configuration - Update these values
KAFKA_BOOTSTRAP_SERVERS = 'localhost:9092'  # Change to your Kafka broker address
KAFKA_TOPIC = 'test-topic'                   # Change to your topic name
KAFKA_GROUP_ID = 'test-consumer-group'       # Consumer group ID

print(f"🔧 Configuration:")
print(f"   - Bootstrap Servers: {KAFKA_BOOTSTRAP_SERVERS}")
print(f"   - Topic: {KAFKA_TOPIC}")
print(f"   - Group ID: {KAFKA_GROUP_ID}")
print()

🔧 Configuration:
   - Bootstrap Servers: localhost:9092
   - Topic: test-topic
   - Group ID: test-consumer-group



In [ ]:
# Create Kafka consumer
try:
    consumer = KafkaConsumer(
        KAFKA_TOPIC,
        bootstrap_servers=KAFKA_BOOTSTRAP_SERVERS,
        group_id=KAFKA_GROUP_ID,
        auto_offset_reset='earliest',  # Start from the beginning
        enable_auto_commit=True,
        value_deserializer=lambda x: x.decode('utf-8'),  # Decode as UTF-8
        consumer_timeout_ms=1000  # 1 second timeout
    )
    
    print("✅ Kafka consumer created successfully!")
    print(f"📡 Connected to: {KAFKA_BOOTSTRAP_SERVERS}")
    print(f"📋 Subscribed to topic: {KAFKA_TOPIC}")
    print()
    
except Exception as e:
    print(f"❌ Error creating Kafka consumer: {str(e)}")
    print("\n🔧 Troubleshooting:")
    print("   1. Check if Kafka broker is running")
    print("   2. Verify the bootstrap servers address")
    print("   3. Ensure the topic exists")
    print("   4. Check network connectivity")
    raise

In [ ]:
# Function to format message for display
def format_message(message):
    """Format a Kafka message for display."""
    timestamp = datetime.fromtimestamp(message.timestamp / 1000.0).strftime('%Y-%m-%d %H:%M:%S')
    
    # Try to parse as JSON, fallback to plain text
    try:
        value = json.loads(message.value)
        value_str = json.dumps(value, indent=2)
    except (json.JSONDecodeError, TypeError):
        value_str = str(message.value)
    
    return f"""
📨 Message Received at {timestamp}
   Topic: {message.topic}
   Partition: {message.partition}
   Offset: {message.offset}
   Key: {message.key.decode('utf-8') if message.key else 'None'}
   Value: {value_str}
   """

In [ ]:
# Start consuming messages
print("🚀 Starting to consume messages...")
print("📝 Messages will be displayed below:")
print("=" * 80)

message_count = 0
start_time = time.time()

try:
    for message in consumer:
        message_count += 1
        
        # Format and display the message
        formatted_message = format_message(message)
        print(formatted_message)
        
        # Print separator
        print("-" * 80)
        
        # Auto-commit offset
        consumer.commit()
        
except KeyboardInterrupt:
    print("\n⏹️  Stopping consumer (KeyboardInterrupt)")
except Exception as e:
    print(f"\n❌ Error consuming messages: {str(e)}")
finally:
    # Close consumer
    consumer.close()
    
    # Print summary
    elapsed_time = time.time() - start_time
    print(f"\n📊 Summary:")
    print(f"   - Total messages received: {message_count}")
    print(f"   - Time elapsed: {elapsed_time:.2f} seconds")
    if elapsed_time > 0:
        print(f"   - Messages per second: {message_count / elapsed_time:.2f}")
    print("✅ Consumer stopped successfully")

In [ ]:
# Alternative: Simple consumer with timeout (for testing)
print("🧪 Testing consumer with 10-second timeout...")
print("📝 This will show any messages received in the next 10 seconds:")
print("=" * 80)

# Create a new consumer for testing
test_consumer = KafkaConsumer(
    KAFKA_TOPIC,
    bootstrap_servers=KAFKA_BOOTSTRAP_SERVERS,
    group_id=f'{KAFKA_GROUP_ID}-test',
    auto_offset_reset='earliest',
    enable_auto_commit=True,
    value_deserializer=lambda x: x.decode('utf-8'),
    consumer_timeout_ms=10000  # 10 second timeout
)

test_message_count = 0
test_start_time = time.time()

try:
    for message in test_consumer:
        test_message_count += 1
        formatted_message = format_message(message)
        print(formatted_message)
        print("-" * 80)
        
except Exception as e:
    if "timeout" in str(e).lower():
        print(f"⏰ No messages received in 10 seconds")
    else:
        print(f"❌ Error: {str(e)}")
finally:
    test_consumer.close()
    
    test_elapsed_time = time.time() - test_start_time
    print(f"\n📊 Test Summary:")
    print(f"   - Messages received: {test_message_count}")
    print(f"   - Time elapsed: {test_elapsed_time:.2f} seconds")
    
    if test_message_count == 0:
        print("\n💡 Tips:")
        print("   - Make sure Kafka broker is running")
        print("   - Verify the topic exists and has messages")
        print("   - Check the bootstrap servers configuration")
        print("   - Try sending some test messages to the topic")

In [ ]:
# Utility: List available topics
print("📋 Available Topics:")
print("=" * 50)

try:
    from kafka.admin import KafkaAdminClient, ConfigResource, ConfigResourceType
    from kafka import KafkaConsumer
    
    # Create admin client
    admin_client = KafkaAdminClient(
        bootstrap_servers=KAFKA_BOOTSTRAP_SERVERS,
        client_id='topic-lister'
    )
    
    # List topics
    topics = admin_client.list_topics()
    
    if topics:
        for topic in sorted(topics):
            print(f"   - {topic}")
    else:
        print("   No topics found")
        
    admin_client.close()
    
except Exception as e:
    print(f"❌ Error listing topics: {str(e)}")
    print("   This might be due to connection issues or permissions")

In [ ]:
# Utility: Test connection
print("🔍 Testing Kafka Connection:")
print("=" * 50)

try:
    # Try to create a simple consumer to test connection
    test_consumer = KafkaConsumer(
        bootstrap_servers=KAFKA_BOOTSTRAP_SERVERS,
        consumer_timeout_ms=5000  # 5 second timeout
    )
    
    # Get cluster metadata
    cluster_metadata = test_consumer.partitions_for_topic('test-topic')
    
    print("✅ Connection successful!")
    print(f"   - Bootstrap servers: {KAFKA_BOOTSTRAP_SERVERS}")
    print(f"   - Cluster metadata available")
    
    test_consumer.close()
    
except Exception as e:
    print(f"❌ Connection failed: {str(e)}")
    print("\n🔧 Troubleshooting:")
    print("   1. Check if Kafka broker is running")
    print("   2. Verify the bootstrap servers address")
    print("   3. Check network connectivity")
    print("   4. Ensure Kafka is accessible from this environment")